In [ ]:
! wget https://www.dropbox.com/s/psutl0zafq50828/data.tsv > ./data.tsv

--2025-02-14 11:35:30--  https://www.dropbox.com/s/psutl0zafq50828/data.tsv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/jvfejz7k8yvr3tobvh0v3/data.tsv?rlkey=qxju3flif8j5bql0gd9prqohg [following]
--2025-02-14 11:35:30--  https://www.dropbox.com/scl/fi/jvfejz7k8yvr3tobvh0v3/data.tsv?rlkey=qxju3flif8j5bql0gd9prqohg
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfc4778c7adc92310fd03252f74.dl.dropboxusercontent.com/cd/0/inline/CkH6G2glrLGeojcSJKex1aef2mEwt_RVe4-jaqP7OGdlxbMug4ugrEsFQFCMdpjHyt2au0aQV8CMwMVsom6uN4M2kWaaDlzohxxIUSi-DyNkhSCLEL6peN3gJmqPVPzV1QPGgCSnWN0_AOoYKpMEK40F/file# [following]
--2025-02-14 11:35:31--  https://ucfc4778c7adc92310fd03252f74.dl.dropboxusercontent.com/cd/0/inline/CkH6G2glrLGeo

In [ ]:
! pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import catboost
import lightgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

data = pd.read_csv('./data.tsv.1', sep='\t')
num_features = ['agent_fee', 'floor', 'floors_total', 'kitchen_area', 'living_area', 'price', 'rooms_offered', 'total_area', 'total_images']
cat_features = ['balcony', 'building_type', 'month', 'renovation', 'studio']

X_train, X_test, y_train, y_test = train_test_split(data[num_features+cat_features], data['exposition_time'], test_size=0.3, shuffle=False)

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
%%time
from catboost import CatBoostRegressor

cbr = CatBoostRegressor(loss_function='MAE', random_seed=0)
cbr.fit(X_train[num_features], y_train)


0:	learn: 38.5557162	total: 88.6ms	remaining: 1m 28s
1:	learn: 38.3751900	total: 124ms	remaining: 1m 1s
2:	learn: 38.2168010	total: 162ms	remaining: 53.9s
3:	learn: 38.0611209	total: 219ms	remaining: 54.5s
4:	learn: 37.9273727	total: 277ms	remaining: 55s
5:	learn: 37.7924777	total: 337ms	remaining: 55.9s
6:	learn: 37.6630257	total: 391ms	remaining: 55.5s
7:	learn: 37.5369072	total: 466ms	remaining: 57.8s
8:	learn: 37.4217519	total: 529ms	remaining: 58.2s
9:	learn: 37.3149421	total: 608ms	remaining: 1m
10:	learn: 37.2036878	total: 685ms	remaining: 1m 1s
11:	learn: 37.1106205	total: 760ms	remaining: 1m 2s
12:	learn: 37.0084084	total: 837ms	remaining: 1m 3s
13:	learn: 36.9218050	total: 904ms	remaining: 1m 3s
14:	learn: 36.8393614	total: 981ms	remaining: 1m 4s
15:	learn: 36.7639278	total: 1.05s	remaining: 1m 4s
16:	learn: 36.6859354	total: 1.13s	remaining: 1m 5s
17:	learn: 36.6095535	total: 1.19s	remaining: 1m 5s
18:	learn: 36.5394863	total: 1.27s	remaining: 1m 5s
19:	learn: 36.4779995	tot

In [ ]:

%%time
lgbm = lightgbm.LGBMRegressor(random_seed=0)
lgbm.fit(X_train[num_features], y_train)



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.017861 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1231
[LightGBM] [Info] Number of data points in the train set: 198390, number of used features: 9
[LightGBM] [Info] Start training from score 51.654625
CPU times: user 1.38 s, sys: 9.78 ms, total: 1.39 s
Wall time: 1.38 s


LGBMRegressor(random_seed=0)

In [ ]:
a = mean_absolute_error(y_test, lgbm.predict(X_test[num_features]))
b = mean_absolute_error(y_test, cbr.predict(X_test[num_features]))
print(round(abs(a - b), 3))

4.864


In [ ]:
param_grid = {
        'learning_rate' : [0.05, 0.1, 0.5],
        'depth' : [5, 7, 9]
}

cbr = CatBoostRegressor(loss_function='MAE', random_seed=0)


In [ ]:
grid_search_result  = cbr.grid_search(param_grid, X_train[num_features], y_train)

Выходные данные были обрезаны до нескольких последних строк (5000).
26:	learn: 35.0572348	test: 34.6919975	best: 34.6919975 (26)	total: 2.49s	remaining: 1m 29s
27:	learn: 35.0269753	test: 34.6680065	best: 34.6680065 (27)	total: 2.6s	remaining: 1m 30s
28:	learn: 35.0058791	test: 34.6498277	best: 34.6498277 (28)	total: 2.7s	remaining: 1m 30s
29:	learn: 34.9917267	test: 34.6392040	best: 34.6392040 (29)	total: 2.77s	remaining: 1m 29s
30:	learn: 34.9750905	test: 34.6275112	best: 34.6275112 (30)	total: 2.86s	remaining: 1m 29s
31:	learn: 34.9621189	test: 34.6185233	best: 34.6185233 (31)	total: 2.94s	remaining: 1m 28s
32:	learn: 34.9414468	test: 34.6062273	best: 34.6062273 (32)	total: 3.01s	remaining: 1m 28s
33:	learn: 34.9273570	test: 34.5974813	best: 34.5974813 (33)	total: 3.11s	remaining: 1m 28s
34:	learn: 34.9152315	test: 34.5895140	best: 34.5895140 (34)	total: 3.2s	remaining: 1m 28s
35:	learn: 34.9041720	test: 34.5843394	best: 34.5843394 (35)	total: 3.25s	remaining: 1m 27s
36:	learn: 34.8

{'params': {'depth': 7, 'learning_rate': 0.05},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
   

In [ ]:
print(round(mean_absolute_error(y_test, cbr.predict(X_test[num_features])), 2))

31.66


In [ ]:
param_grid = {
        'learning_rate' : [0.05, 0.1, 0.5],
        'depth' : [5, 7, 9]
}

cbr = CatBoostRegressor(loss_function='MAE', random_seed=0)
grid_search_result  = cbr.grid_search(param_grid, X_train[num_features], y_train)

cbr2 = CatBoostRegressor(loss_function='MAE', random_seed=0, cat_features = cat_features)
grid_search_result1  = cbr2.grid_search(param_grid, X_train, y_train)

Выходные данные были обрезаны до нескольких последних строк (5000).
271:	learn: 34.4263367	test: 34.4931958	best: 34.4931958 (271)	total: 14.5s	remaining: 38.8s
272:	learn: 34.4242931	test: 34.4936673	best: 34.4931958 (271)	total: 14.6s	remaining: 38.8s
273:	learn: 34.4221079	test: 34.4942047	best: 34.4931958 (271)	total: 14.6s	remaining: 38.8s
274:	learn: 34.4211370	test: 34.4941899	best: 34.4931958 (271)	total: 14.7s	remaining: 38.8s
275:	learn: 34.4190477	test: 34.4928913	best: 34.4928913 (275)	total: 14.8s	remaining: 38.8s
276:	learn: 34.4173552	test: 34.4927617	best: 34.4927617 (276)	total: 14.9s	remaining: 38.8s
277:	learn: 34.4129028	test: 34.4931341	best: 34.4927617 (276)	total: 14.9s	remaining: 38.8s
278:	learn: 34.4119416	test: 34.4931358	best: 34.4927617 (276)	total: 15s	remaining: 38.7s
279:	learn: 34.4103187	test: 34.4933279	best: 34.4927617 (276)	total: 15s	remaining: 38.6s
280:	learn: 34.4080850	test: 34.4929330	best: 34.4927617 (276)	total: 15.1s	remaining: 38.5s
281:	l

KeyboardInterrupt: 

In [ ]:
prd1 =cbr.predict(X_train[num_features])
prd2 = cbr2.predict(X_train)

34.7333


In [ ]:
import numpy as np
def select_weights(y_true, y_pred_1, y_pred_2):
    metric = []
    grid = np.linspace(0, 1, 100)
    grid2 = np.linspace(0, 1, 100)
    for w_0 in grid:
      for w_1 in grid2:
        #w_1 = 1 - w_0
        y = y_pred_1 * w_0 + y_pred_2 * w_1

        metric.append([mean_absolute_error(y_true, y), w_0, w_1])

    return metric

In [ ]:
rmse_blending_train, w_0, w_1 = min(select_weights(y_train, prd1, prd2), key=lambda x: x[0])
rmse_blending_train
print(w_0, w_1)
#rmse_blending_train, w_0, w_1 = min(select_weights(y_train, y_train_pred_cbm, y_train_pred_lr), key=lambda x: x[0])

0.0 1.0


In [ ]:
print("Test RMSE blending = %.4f" % mean_absolute_error(y_test, cbr.predict(X_test[num_features]) * w_0 +  cbr2.predict(X_test) * w_1))

Test RMSE blending = 34.7333
